In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nose.tools import *

np.random.seed(24680)

Write your imports in the cell below.

In [3]:
# YOUR CODE HERE
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.svm import LinearSVC ,SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,make_scorer

# Ensemble Models and Support Vector Machines Lab
## Training and comparing different algorithms

Once again, we'll work with the bank dataset. This time, the data preprocessing steps have been done for you.

The goal is to try and improve our predictions (if they can be improved at all) using different types of algorithms.

### 1. Read the data (1 point)
This time you only need to read the data. The indicator variables have been separated out for you.

Read the dataset and save it in the variable `bank_data`. The target column is `y`. Use the variables `bank_attributes` and `bank_labels` to save the attributes (explanatory variables, features, predictors), and labels (`y`).

In [4]:
bank_data = pd.read_csv("data/bank.csv")
bank_attributes = bank_data.drop(columns = ["y"])
bank_labels = bank_data["y"]
# YOUR CODE HERE

In [5]:
assert_is_not_none(bank_data)
assert_is_not_none(bank_attributes)
assert_is_not_none(bank_labels)

### 2. Normalize the data (1 point)
Because both forests and SVMs are sensitive to non-scaled data, we need to normalize our dataset first.

Rescale all columns in `bank_attributes` so they have mean = 0 and variance = 1. You can either look at the `sklearn` docs or do this yourself. When you're ready, overwrite the `bank_attributes` column. Make sure that you don't lose the column names in the process.

In [6]:
# YOUR CODE HERE
column_names = bank_attributes.columns
bank_attributes = MinMaxScaler().fit_transform(bank_attributes)
bank_attributes = pd.DataFrame(bank_attributes)
bank_attributes.columns = column_names

In [7]:
assert_is_not_none(bank_attributes)

### 3. Split the data (1 point)
Use the standard 70% / 30% split. Since this is a classification problem, be sure to stratify the split according to the `bank_labels`.

In [8]:
bank_attributes_train,bank_attributes_test, bank_labels_train,bank_labels_test = train_test_split(bank_attributes,bank_labels,stratify = bank_labels,test_size = 0.3,shuffle = True)
# YOUR CODE HERE

In [9]:
(bank_attributes_train.shape,bank_attributes_test.shape, bank_labels_train.shape,bank_labels_test.shape)

((3164, 51), (1357, 51), (3164,), (1357,))

In [10]:
assert_is_not_none(bank_attributes_train)
assert_is_not_none(bank_labels_train)

assert_is_not_none(bank_attributes_test)
assert_is_not_none(bank_labels_test)

### 4. Prepare the cross-validation folds (1 point)
Use a stratified k-fold cross-validation split, with $k = 5$. Fit it to the train data. Save the trained cross-validator to the variable `k_fold`.

The data should already be shuffled. There's no need to shuffle it again.

In [11]:
k_fold = KFold(n_splits = 5)
# GridSearchCV(estimator=svm.SVC(max_iter=1000),param_grid={
#     'C':[0.01,0.1,1,10,100],
#     'loss':['hinge','squared_hinge']
# },cv=k_fold)
# YOUR CODE HERE

In [12]:
assert_is_not_none(k_fold)

### 5. Decision Tree (2 points)
Use cross-validation to train and optimize the hyperparameters for a decision tree classifier.

Use grid search with the following grid:
* `max_depth`: 1, 5, 7, 15, 20
* `min_samples_leaf`: 2, 5, 10, 12
* `max_leaf_nodes`: 5, 10, 20

Use the most appropriate scoring metric (remember that accuracy doesn't work in this case because the data is highly imbalanced; we need something which combines precision and recall). Use the cross-validation splits you just created.

Save the grid results in `grid_search`. Save the best classifier in `tree_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [13]:
grid_search =GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = {
    "max_depth": [1, 5, 7, 15, 20],
    'min_samples_leaf': [2, 5, 10, 12],
    'max_leaf_nodes': [5, 10, 20]
}, 
                          cv = k_fold,scoring=make_scorer(f1_score))
grid_search.fit(bank_attributes_train,bank_labels_train)
tree_classifier =grid_search.best_estimator_

# YOUR CODE HERE


print("Decision tree; best score:", grid_search.best_score_)

Decision tree; best score: 0.48698726126425945


In [14]:
tree_classifier

DecisionTreeClassifier(max_depth=7, max_leaf_nodes=10, min_samples_leaf=2)

In [15]:
assert_is_not_none(grid_search)
assert_is_not_none(tree_classifier)

### 6. Random Forest (1 point)
Use cross-validation to train and optimize the hyperparameters for a random forest classifier. Use the same technique as before.

Use the following grid:
* `n_estimators`: 100, 200, 300 
* `max_depth`: 20, 50, 100

Note that this grid is on the small side but this is mainly due to performance reasons. Also note that the training will take some time.

Save the grid results in `grid_search`. Save the best classifier in `forest_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

Due to the relatively slow training, we've chosen low values for the parameters. The performance of the random forest will be worse than the decision tree. This is not necessarily the case in general, it's due to the parameters we've chosen to try.

In [16]:
grid_search =GridSearchCV(estimator = RandomForestClassifier(), param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [20, 50, 100]
}, 
                          cv = k_fold,scoring=make_scorer(f1_score))
grid_search.fit(bank_attributes_train,bank_labels_train)
forest_classifier = grid_search.best_estimator_

# YOUR CODE HERE

print("Random forest; best score:", grid_search.best_score_)

Random forest; best score: 0.3865169599140891


In [17]:
assert_is_not_none(grid_search)
assert_is_not_none(forest_classifier)

### 7. Linear SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for a linear support vector machine. Use the same technique as before.

Use the following grid:
* `C`: 0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20

Note that we're choosing relatively small values for `C`. This is allowed because our data is normalized.

Save the grid results in `grid_search`. Save the best classifier in `linear_svm_classifier`. There are many ways to create a linear SVM classifier. Look up the `sklearn` docs to choose the fastest one (in terms of performance).

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [18]:
grid_search =GridSearchCV(estimator = LinearSVC(), param_grid = {
    'C': [ 0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20]
}, 
                          cv = k_fold,scoring=make_scorer(f1_score))
grid_search.fit(bank_attributes_train,bank_labels_train)
linear_svm_classifier = grid_search.best_estimator_

# YOUR CODE HERE
print("Linear SVM; best score:", grid_search.best_score_)

C:\Users\Tihomir111\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Tihomir111\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Tihomir111\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Tihomir111\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Tihomir111\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the numb

Linear SVM; best score: 0.3964890331597575


C:\Users\Tihomir111\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [19]:
assert_is_not_none(grid_search)
assert_is_not_none(linear_svm_classifier)

### 8. Gaussian SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for an SVM with a Gaussian kernel. Use the same technique as before.

Use the following grid:
* `C`: 10, 15, 20, 50, 200
* `gamma`: 0.001, 0.01, 0.1, 0.2

Note that this time we give larger values of `C` because the governing parameter here is `gamma`.

Save the grid results in `grid_search`. Save the best classifier in `gaussian_svm_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [20]:
grid_search =GridSearchCV(estimator = SVC(), param_grid = {
    'C': [10, 15, 20, 50, 200],
    'gamma': [0.001, 0.01, 0.1, 0.2]
}, 
                          cv = k_fold,scoring=make_scorer(f1_score))
grid_search.fit(bank_attributes_train,bank_labels_train)
gaussian_svm_classifier = grid_search.best_estimator_

# YOUR CODE HERE
print("Gaussian SVM; best score:", grid_search.best_score_)

Gaussian SVM; best score: 0.4290787528000507


In [21]:
assert_is_not_none(grid_search)
assert_is_not_none(gaussian_svm_classifier)

### 9. Compare performance on the testing data (1 point)
Now that you've trained all your models, you've got to select the best one. This should be done on the testing data.

Use the appropriate scoring metric to get the testing scores for all your models. Don't forget to pass the **testing**, not the training data. Save all scores.

Choose the best classifier, based on these scores (the one with the highest test score). Of course, this is not enough. We need to look at ROC curves, track performance through other measures, debug the sources of variance in testing results, try more hyperparameters, etc. However, this is enough for an introductory lab :).

Optionally, you can think of combining them into a boosted model but this is out of the scope of this lab.

In [22]:
tree_classifier_score, forest_classifier_score, linear_svm_classifier_score, gaussian_svm_classifier_score = (f1_score(bank_labels_test, tree_classifier.predict(bank_attributes_test)), 
f1_score(bank_labels_test, forest_classifier.predict(bank_attributes_test)), 
f1_score(bank_labels_test, linear_svm_classifier.predict(bank_attributes_test)), 
f1_score(bank_labels_test, gaussian_svm_classifier.predict(bank_attributes_test)), 
)
# YOUR CODE HERE

print("Testing scores:")
print("Decision tree:", tree_classifier_score)
print("Random forest:", forest_classifier_score)
print("Linear SVM:", linear_svm_classifier_score)
print("Gaussian SVM:", gaussian_svm_classifier_score)

Testing scores:
Decision tree: 0.4638783269961977
Random forest: 0.3348837209302326
Linear SVM: 0.4052863436123348
Gaussian SVM: 0.4107744107744108


In [23]:
best_classifier = "Linear SVM" # Replace empty string with "tree", "forest", "linear SVM" or "gaussian SVM"
# YOUR CODE HERE

In [24]:
assert_not_equal(best_classifier, "")